In [1]:
///\section*{What's new:} 
///\begin{itemize}
///    \item{generating .plt-files for visualization}
///    \item{implementing a numerical flux}
///    \item{definition and evaluation of the spatial operator}
///    \item{explicit time integration}
///\end{itemize}
///\section*{Prerequisites:} 
///\begin{itemize}
///    \item{projection onto a DG-field}
///\end{itemize} 
///Within this tutorial, we are going to implement the scalar transport equation via the definition of a spatial operator and an explicit time integrator. The implementation of the numerical flux is described on the basis of a upwinding scheme. For the visualization of the results, we are generating .plt-files, which can be opened by a viewer of your choice
///\section{Problem statement}
///We are considering the following definition of the scalar transport equation with
///\begin{equation}
///   \label{eq:divergenceTerm}
///   \frac{\partial c}{\partial t} + \nabla \cdot (\vec{u} c) = 0,
///\end{equation}
///where $c = c(x,y,t) \in \mathbb{R}$ is the unknown concentration and
///\begin{equation*}
///     \vec{u} = \begin{pmatrix}
///         y\\-x
///     \end{pmatrix}
///\end{equation*}
///is a given velocity field in $\domain = [-1, 1] \times [-1, 1]$. Furthermore, the exact solution is given by
///\begin{equation*}
///    c_\text{Exact}(x,y,t) = \cos(\cos(t) x - \sin(t) y) \quad \text{ for } (x,y) \in \domain
///\end{equation*}
///In this tutorial we will simulate the evolution of the concentration $c(x,y,t)$.
///\section{Solution within the \BoSSS{} framework}
///We start a new project

The below script needs to be able to find the current output cell; this is an easy method to get it.

In [2]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


In [3]:
using System.IO;

In [4]:
/// Delete old plots in the current directory if any
Directory.GetFiles(".", "*.plt").ForEach(file => File.Delete(file));

In [5]:
/// \subsection{Projection and visualization}
/// In this first section we get to know the plotting tool \code{Tecplot}, 
/// which generates .plt-files of our \code{DGFields}. 
/// Previously, we define the exact solution $c_{Exact}(x,y,t)$ and 
/// the scalar components of the velocity field $\vec{u}$ as functions:

In [6]:
public static class ExactSol {
    public static double c(double[] X, double t) => Math.Cos(Math.Cos(t)*X[0] - Math.Sin(t)*X[1]);
}

In [7]:
public static class VelField {
    public static double u(double[] X) => X[1];
    public static double v(double[] X) => -X[0];
}

In [8]:
/// Next, we need to construct the computational domain, i.e a unit square with one cell.
double[] nodes = GenericBlas.Linspace(-1.0, 1.0, 2); 
GridCommons grid = Grid2D.Cartesian2DGrid(nodes, nodes); 

In [9]:
/// We instantiate the \code{SinglePhaseField} \emph{ch} with a \code{Basis} of DG-degreee of 2. Then we can project the initial value $c(x,y,0.0)$ onto \emph{ch}.
int dgDegree = 2;  
Basis basis = new Basis(grid, dgDegree);  
SinglePhaseField ch = new SinglePhaseField(basis, "ch");  
ch.ProjectField(X => ExactSol.c(X, 0.0));

In [10]:
/// Now, we can export the initial projection in our \code{Tecplot} format.
using BoSSS.Solution.Tecplot;

In [11]:
/// One important parameter for visualization is \textit{superSampling}. It is essential for higher order methods since almost all
/// plotting tools work with piecewise linear interpolations of the data in the vertices. For our
/// case, the plot with \code{superSampling=0} would just show a constant value! By increasing the
/// rate of the \emph{superSampling}, we provide more sampling points for the plot tool.
/// \begin{itemize}
/// \item This has nothing to do with the computation! Only required for visualization!
/// \item The number of sampling points grows exponentially with the value of
///  \code{superSampling}. Never use a value above 5 or 6!
/// \end{itemize}

In [12]:
uint superSampling = 0;  
Tecplot tecplot    = new Tecplot(grid.GridData, superSampling);
//tecplot.PlotFields( 
//    "plot_tutorial4_superSampling0", 
//    0.0, 
//    ch);

In [13]:
superSampling = 3;  
tecplot    = new Tecplot(grid.GridData, superSampling); 
//tecplot.PlotFields( 
//    "plot_tutorial4_superSampling3", 
//    0.0, 
//    ch);

In [14]:
/// There should now be two plot-files in your current directory. Those can be opened by any standard viewer for .plt-files.

In [15]:
/// \subsection{Implementation of the (numerical) flux}
/// Before we can define the spatial operator for the scalar transport equation, we need to implement 
/// the flux for the given problem, i.e. the flux of the divergence operator. A flux defines the volume term 
/// (the \emph{flux}) as well as the boundary terms (via the \emph{numerical flux}).
/// We derive such a flux from the class \code{NonlinearFlux} which simplifies the implementation of fluxes in
/// flux-based formulations. 
using BoSSS.Platform.LinAlg;  
class ScalarTransportFlux : NonlinearFlux {  
 
    /// \leftskip=1cm \code{ArgumentOrdering} defines on which arguments this flux depends, and in which order. 
    /// Here, we have just one argument (the concentration $c$). The name is arbitrary at this point, but has to be 
    /// referenced when defining the spatial operator (see next section). Since this flux only depends on one
    /// argument, the parameters \code{U}, \code{Uin} and \code{Uout} will have a length of 1 and will contain the
    /// local values of $c$ in the first entry, i.e. \code{U[0]} in the method \code{Flux(...)}
    public override IList<string> ArgumentOrdering {  
        get { return new string[] { "c" }; }  
    }  
 
    /// \code{Flux(...)} defines the volume term. The array \code{output} (whose length is determined by the
    /// spatial dimension of the problem) has to contain the evaluated \emph{flux} on exit.
    protected override void Flux(double time, double[] x, double[] U, double[] output) {  
        output[0] = VelField.u(x) * U[0];  
        output[1] = VelField.v(x) * U[0];  
    }  
 
    /// \code{InnerEdgeFlux(...)} defines the \emph{numerical flux} between inner edges. The parameters 
    /// \code{Uin} and \code{Uout} contain the value from the \emph{in} and \emph{out} side, respectively, 
    /// where the normal vector \code{normal} points from \emph{in} to \emph{out}
    protected override double InnerEdgeFlux(double time, double[] x, double[] normal, 
    double[] Uin, double[] Uout, int jEdge) {  
        Vector n              = new Vector(normal);  
        Vector velocityVector = new Vector( VelField.u(x),  VelField.v(x));  
 
        if (velocityVector * n > 0) {  
            return (velocityVector * Uin[0]) * n;  
        } else {  
            return (velocityVector * Uout[0]) * n;  
        }  
    }  
 
    /// \code{BorderEdgeFlux(...)} defines the \emph{numerical flux} at boundary edges, where only inner values 
    /// (\code{Uin}) are given. Here, we reuse \code{InnerEdgeFlux(...)} and the exact solution \code{cExact} 
    /// to define a suitable boundary condition.
    protected override double BorderEdgeFlux(double time, double[] x, double[] normal, 
                                             byte EdgeTag, double[] Uin, int jEdge) {  
        double[] Uout = new double[] { ExactSol.c(x, time) };  
        return InnerEdgeFlux(time, x, normal, Uin, Uout, jEdge);  
    }  
} 
/// \leftskip=0cm

In [16]:
/// \subsection{Definition of a spatial operator}
/// The next step is the definition of the spatial operator.
var spatialTerm = new SpatialOperator( 
    new string[] { "c" },        // Domain variable 
    new string[] { "div" },      // Co-domain variable
    QuadOrderFunc.NonLinear(2)); // Order of integration
/// The name of the \emph{domain variable} must be the same used in \code{ArgumentOrdering}
/// in the definition of the flux, i.e \code{ScalarTransportFlux}.
/// The name of the \emph{co-domain variable} is arbitrary and is used when the 
/// fluxes are added. In our case, we only have one type of flux.
/// \code{QuadOrderFunc.NonLinear(int x)} computes the required integration order 
/// for a non-linear flux. Here, the flux is given by $\vec{u} c$, where $\vec{u}$ is linear. 
/// So, we have second order terms (flux times the ansatz functions), i.e the required
/// order is \code{2*dgDegree+1} \newline

In [17]:
/// We add the flux of the divergence operator to the corresponding equation component, identified by the 
/// \emph{co-domain variable},
spatialTerm.EquationComponents["div"].Add(new ScalarTransportFlux());

In [18]:
/// and finalize the definition of the operator
spatialTerm.Commit();

In [19]:
/// \subsection{Time integration}
/// Finally, we have to specify a time stepping scheme to solve the time dependent
/// scalar transport equation \eqref{eq:divergenceTerm}. For simplification, we use 
/// the \code{ExplicitEuler} scheme, which just needs the \code{SpatialOperator} and 
/// the \code{DGField} as arguments.
using BoSSS.Solution.Timestepping;

In [20]:
ExplicitEuler timeStepper = new ExplicitEuler(spatialTerm, ch);

In [21]:
/// We want to perform a full revolution ($t \in [0, 2\pi]$) with 24 timesteps.

In [22]:
double endTime = 2.0 * Math.PI; 
int numberOfTimesteps = 24;

In [23]:
/// Plot the initial data
tecplot.PlotFields( 
    "plot_tutorial4_0", 
    0.0, 
    ch); 
/// Now, we can start the simulation, where the \code{timestepper} performs in each iteration one
/// explicit euler timestep with the timestep size \code{dt}
double dt = endTime / numberOfTimesteps; 
for (int i = 1; i <= numberOfTimesteps; i++) { 
    timeStepper.Perform(dt); 
    tecplot.PlotFields(          // plot each timestep
        "plot_tutorial4_" + i, 
        timeStepper.Time, 
        ch); 
}

In [24]:
/// Furthermore, we can postprocess our simulation data in various ways: For example, 
/// we can compute the L2-Error at the end of the simulation
double error = ch.L2Error(X => ExactSol.c(X, timeStepper.Time)); 
error

0.10126945730309017

In [25]:
/// \section{Advanced topics}
/// So far we used the \code{timestepper} to evaluate the \code{SpatialOperator} in order to get 
/// the current change rate for the explicit Euler scheme. In the next section we will evaluate the operator 
/// in each iteration manually. But first we set the \code{DGField} back to the initial values and plot
ch.ProjectField(X => ExactSol.c(X, 0.0));
tecplot.PlotFields( 
    "plot_tutorial4_advanced_0", 
    0.0, 
    ch);

In [26]:
/// \subsection{Evaluation of the spatial operator}
/// To evaluate the \code{SpatialOperator}, we have to provide a mapping of the DG-coordinates of \emph{ch}.
/// This describes a bijective mapping between \emph{local unique indices} and \emph{global unique indices}
var mapping = new CoordinateMapping(ch);

In [27]:
/// In other words, it maps the DG-coordinates into one long, one-dimensional \code{CoordinateVector}
var DGCoordinates = new CoordinateVector(mapping);

In [28]:
/// Now, we can create an evaluator of the \code{SpatialOperator} for the \code{DGField} \emph{ch} 
var evaluator = spatialTerm.GetEvaluatorEx(mapping.Fields, null, mapping);

In [29]:
/// In our case this evaluator computes the fluxes of the divergence operator evaluated for the \code{DGField} \emph{ch}
double[] flux = new double[ch.CoordinateVector.Count]; 
evaluator.time = 0.0;
evaluator.Evaluate(1.0, 0.0, flux);
/// After the evaluation the output \code{flux} is $\code{flux} = 0.0 \cdot \code{flux} + 1.0 \cdot \code{spatialTerm(time: 0.0)}$

In [30]:
/// Finally, we can use this kind of \code{spatial operator} evaluation to implement the explicit Euler scheme manually.   
double physTime = 0.0;
for (int i = 1; i <= numberOfTimesteps; i++) { 
    evaluator.time = physTime;
    evaluator.Evaluate(1.0, 0.0, flux);
    DGCoordinates.axpy<double[]>(flux, -dt);
    physTime += dt;
    tecplot.PlotFields(        
        "plot_tutorial4_advanced" + i, 
        physTime, 
        ch); 
}

In [31]:
double error = ch.L2Error(X => ExactSol.c(X, physTime)); 
error

0.10126945730309017